In [ ]:
import re
import glob
from google.colab import runtime
import codecs
from io import StringIO
import pandas as pd

def format_text_file(file_path):

  #変数の初期化
  bits = []
  four_byte = []
  data=[]
  text = ""
  memory = []

  #テキストファイルを開く
  with codecs.open(file_path, 'r',encoding = "shift-jis",errors='ignore') as f:

    for _ in range(8):
      f.readline()

    while True:

        text = f.readline()
        memory.append(text)
        bits = []


        if "64 61 74 61" in text:
          continue

        elif text in memory[:len(memory)-1]:
          break

        else:

          if len(text) < 65:
            break

          else:
            #バイト列の取得
            text = text[9:32]+text[34:58]
            text = re.sub(r'[\n\r]', '', text)

            for i in text.split():
              bits.append(i)

            #リトルエンディアンの変換

            for i in range(0,len(bits),2):
              four_byte.append(bits[i+1]+bits[i])

  f.close()
  #print(four_byte)
  return four_byte

def hex_to_signed_int(hex_str, bits=16):
    # 16進数文字列を10進数の整数に変換
    unsigned_int = int(hex_str, 16)

    # 符号ビットを考慮して符号付き整数に変換
    if unsigned_int >= 2**(bits - 1):
        signed_int = unsigned_int - 2**bits
    else:
        signed_int = unsigned_int

    return signed_int

#テキストファイルの総取得
data = glob.glob('/content/*.txt')

hz = int(input("信号の周波数を入力してください\n(例: 200)"))

if "/content/output.txt" in data:
  #出力は配列から除く
  data.remove("/content/output.txt")

#0以下または2つ以上ファイルがあるときの例外処理，
if len(data) == 0:
  print("No text files found.")


elif len(data) > 1:
  print(data)
  print("More than one text file found.")

else:
  #処理の実行
  hexs = format_text_file(data[0])
  nums = list(map(lambda x:hex_to_signed_int(x,16),hexs))
  num_start = nums[0]

  cnt = 0
  size = 0

  for i,num in enumerate(nums):
    #print(i,num,cnt,num_start)
    if num_start == num :
      cnt += 1
    if cnt == 2:
      size = i
      break

  #print(size)

  output = nums[:size]

  #出力
  f = open('/content/output.txt',"w")


  print(*output,file=f)
  #print(nums)
  print(*output)
  print(len(output))
  print("サンプリング周波数は{}[Hz]です".format(hz*len(output)))

  f.close()


信号の周波数を入力してください
(例: 200)1000
16384 11585 0 -11585 -16384 -11585 0 11585
8
サンプリング周波数は8000[Hz]です
